# Analysis

Using the data obtained from the SQL server by using the queries listed in survey_analysis.sql and analysis_RQ2.sql, we can now answer our second research question.

In [9]:
import pandas as pd 
import altair as alt
import numpy as np

In [10]:
# reading data from csv file produced by survey_expenditure.sql and analysis_RQ2.sql
analysis_df = pd.read_csv('../Data/survey_analysis.csv')

analysis_df

,province,per_cap_spend,coordination,waiting_time,care_access,provider_access,unmet_needs
0,Alberta,5317.510,0.57950,0.666000,0.904,0.873,NaN
1,BritishColumbia,4762.885,0.54200,0.626167,0.895,0.849,0.945
2,Man.,5021.065,0.60050,0.626667,0.897,0.888,0.966
3,N.B.,4593.585,0.67100,0.511500,0.926,0.928,0.962
4,N.L.,6319.605,0.67675,0.565667,0.949,0.895,0.965
5,N.S.,5121.230,0.62500,0.498167,0.908,0.887,0.935
6,Ontario,4610.650,0.58750,0.639333,0.927,0.913,0.952
7,P.E.I.,5101.495,0.68225,0.610667,0.929,0.910,NaN
8,Quebec,5037.435,0.55725,0.531833,0.849,0.830,0.953
9,Sask.,5198.730,0.61450,0.670500,0.907,0.875,NaN


Now that we have a dataset that only includes our key variables which are the province, per capita spending, and normalized measures of each relevant survey question, we can work towards answering our question. To start with, we'll take the simple approach and just aggregate all our quality variables into a single column and find the care quality to per capita expenditure for each province.

In [ ]:

# make a new column showing the mean of all quality variables for each row
analysis_df['care_quality'] = analysis_df[['coordination','waiting_time','care_access','provider_access',
                                        'unmet_needs']].mean(axis=1)

# make a new column created by adjusting care_quality based on per capita spending
analysis_df['efficiency'] = analysis_df.apply(lambda row: 10000*(row.care_quality/row.per_cap_spend), axis=1)

# sort dataframe so the highest efficiency values appear first
analysis_df = analysis_df.sort_values(by='efficiency',ascending=False)

analysis_df

,province,per_cap_spend,coordination,waiting_time,care_access,provider_access,unmet_needs,care_quality,efficiency
6,Ontario,4610.650,0.58750,0.639333,0.927,0.913,0.952,0.803767,1.743283
3,N.B.,4593.585,0.67100,0.511500,0.926,0.928,0.962,0.799700,1.740906
1,BritishColumbia,4762.885,0.54200,0.626167,0.895,0.849,0.945,0.771433,1.619677
2,Man.,5021.065,0.60050,0.626667,0.897,0.888,0.966,0.795633,1.584591
7,P.E.I.,5101.495,0.68225,0.610667,0.929,0.910,NaN,0.782979,1.534803
5,N.S.,5121.230,0.62500,0.498167,0.908,0.887,0.935,0.770633,1.504782
8,Quebec,5037.435,0.55725,0.531833,0.849,0.830,0.953,0.744217,1.477372
9,Sask.,5198.730,0.61450,0.670500,0.907,0.875,NaN,0.766750,1.474879
0,Alberta,5317.510,0.57950,0.666000,0.904,0.873,NaN,0.755625,1.421013
4,N.L.,6319.605,0.67675,0.565667,0.949,0.895,0.965,0.810283,1.282174


Based on these preliminary results, we already have some interesting results. First, Ontario comes out on top which isn't very surprising as it is the largest province by population and home to the federal government. As such, it benefits by being the focus of the government and enjoys economies of scale. However, by that logic we would also expect Quebec to be near the top, but it is actually in the bottom half of the table. 

New Brunswick is the real surprise winner here, just barely losing first place to Ontario by a fraction of a percent. Without any obvious external advantages present, this definitely suggests that New Brunswick is doing something right. British Columbia is another large, well supported province we would expect to see high up on the list. The only other notable outliers on this list are Alberta and the Territories. Alberta should be up near the top with Ontario and BC, but here it is at the third from the bottom, a surprisingly poor result. 

However, this may be because Alberta is missing results from the unmet needs survey question but Prince Edward Island also lacks these results and still placed above two provinces that have it, including Quebec which suggests it may not be a major disrupting factor. Finally, the territories are a massive outlier with less than half the efficiency score of the lowest-ranking province. Unfortunately this is not very surprising, as the geographic challenges and low population density of the territories make providing effective healthcare incredibly difficult.

Technically, this answers our research question but we'll do some additional analysis for extra measure

In [16]:
alt.Chart(analysis_df).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='province:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['coordination','waiting_time','care_access','provider_access','unmet_needs'],
    column=['unmet_needs','provider_access','care_access', 'waiting_time', 'coordination']
).interactive()

alt.RepeatChart(...)